# Analyzing ACS 2017 and standardized decennial Census data from 2000

This script combines and analyzes data from the 5-year estimates of the American Community Survey (2013-2017) and standardized decennial Census data from the [US2010 Longitudinal Tract Data Base](https://s4.ad.brown.edu/Projects/Diversity/Researcher/Bridging.htm) to determine the following:
* Whether a tract gentrified or not according to [this methodology](http://www.governing.com/gov-data/gentrification-report-methodology.html)
* Percent-point changes for six non-overlapping race/ethnicity groups

In [1]:
import pandas as pd
import geopandas as gp
import numpy as np

#### Load data and filter it down to metro level

* Load data from the [Longitudinal Tract Data Base](https://s4.ad.brown.edu/Projects/Diversity/Researcher/Bridging.htm). (Column definitions are specified in the LTDB"s [data dictionary](https://s4.ad.brown.edu/Projects/Diversity/Researcher/LTBDDload/Dfiles/codebooks.pdf).)
* Load data from 2017 ACS


In [2]:
CSV_OPTIONS_2000 = dict(
    encoding = "ISO-8859-1",
    dtype = {
        "state": str,
        "county":str,
        "tract": str,
        "TRTID10": str,
        "MHMVAL00": float,
        "HINC00": float
    },
    na_values = [ " " ]
)

In [3]:
full_count_data = (
    pd.read_csv(
        "../data/LTDB_Std_2000_fullcount.csv",
        **CSV_OPTIONS_2000
    )
    [[
        "TRTID10",
        "state", 
        "county", 
        "tract",
        "POP00",
        "NHWHT00",
        "NHBLK00",
        "NTV00",
        "ASIAN00",
        "HAW00",
        "HISP00",
    ]]
    .assign(
        TRTID10 = lambda df: df["TRTID10"].str.zfill(11)
    )
)

full_count_data.head()

,TRTID10,state,county,tract,POP00,NHWHT00,NHBLK00,NTV00,ASIAN00,HAW00,HISP00
0,01001020100,AL,Autauga County,Census Tract 201,1920.974695,1722.977295,144.998093,28.999619,7.999895,0.0,11.999842
1,06083002402,CA,Santa Barbara County,Census Tract 24.02,6742.000000,1573.000000,126.000000,44.000000,272.000000,11.0,4700.000000
2,01001020200,AL,Autauga County,Census Tract 202,1892.000000,671.000000,1177.000000,12.000000,12.000000,0.0,16.000000
3,06083002102,CA,Santa Barbara County,Census Tract 21.02,2041.000000,1204.000000,57.000000,19.000000,60.000000,2.0,696.000000
4,01001020300,AL,Autauga County,Census Tract 203,3339.000000,2738.000000,498.000000,16.000000,27.000000,1.0,55.000000


In [4]:
full_count_data.dtypes

TRTID10     object
state       object
county      object
tract       object
POP00      float64
NHWHT00    float64
NHBLK00    float64
NTV00      float64
ASIAN00    float64
HAW00      float64
HISP00     float64
dtype: object

In [5]:
len(full_count_data)

72693

In [6]:
sample_data = pd.read_csv(
    "../data/LTDB_Std_2000_Sample.csv",
    **CSV_OPTIONS_2000,
)

sample_data = sample_data[[
    "TRTID10", 
    "state", 
    "county", 
    "tract",
    "AG25UP00",
    "HINC00", 
    "MHMVAL00", 
    "COL00"
]]

sample_data.head()

,TRTID10,state,county,tract,AG25UP00,HINC00,MHMVAL00,COL00
0,01001020100,AL,Autauga County,Census Tract 201,1226.983887,36685.00080,76600.00185,191.997467
1,01001020200,AL,Autauga County,Census Tract 202,1157.000000,30298.00000,72900.00000,170.000000
2,01001020300,AL,Autauga County,Census Tract 203,2130.000000,46731.00000,79900.00000,478.000000
3,01001020400,AL,Autauga County,Census Tract 204,3072.000000,46142.00000,89800.00000,708.000000
4,01001020500,AL,Autauga County,Census Tract 205,3785.214923,58886.26948,116593.52740,1214.091953


In [7]:
census_data_2000 = (
    pd.merge(
        full_count_data,
        sample_data,
        on = [ "state", "county", "tract", "TRTID10" ],
        how = "left",
        validate = "1:1",
    )
    .rename(columns = {
        "TRTID10": "GEOID", 
        "POP00": "total_population", # total population
        "AG25UP00": "total_population_25_over", # total population 25 and over
        "NHWHT00": "white_alone", # persons of white race, not Hispanic origin -- equivalent of 2010 "white alone"
        "NHBLK00":"black_alone", # persons of black race, not Hispanic origin -- equivalent of 2010 "black alone"
        "NTV00": "native_alone", #  persons of Native American race
        "ASIAN00": "asian_alone", # Asian and Pacific Islander race
        "HAW00": "native_hawaiian_pacific_islander",  # persons of Hawaiian race
        "HISP00":"hispanic_or_latino", # persons of Hispanic origin 
        "HINC00": "median_income", # Median household income, total
        "MHMVAL00": "median_home_value", # Median home value
        "COL00": "educational_attainment" # persons with at least a four-year college degree
    })
)

print(len(census_data_2000), len(sample_data), len(full_count_data))
census_data_2000.head()

72693 72693 72693


,GEOID,state,county,tract,total_population,white_alone,black_alone,native_alone,asian_alone,native_hawaiian_pacific_islander,hispanic_or_latino,total_population_25_over,median_income,median_home_value,educational_attainment
0,01001020100,AL,Autauga County,Census Tract 201,1920.974695,1722.977295,144.998093,28.999619,7.999895,0.0,11.999842,1226.983887,36685.0008,76600.00185,191.997467
1,06083002402,CA,Santa Barbara County,Census Tract 24.02,6742.000000,1573.000000,126.000000,44.000000,272.000000,11.0,4700.000000,3298.000000,31803.0000,127000.00000,205.000000
2,01001020200,AL,Autauga County,Census Tract 202,1892.000000,671.000000,1177.000000,12.000000,12.000000,0.0,16.000000,1157.000000,30298.0000,72900.00000,170.000000
3,06083002102,CA,Santa Barbara County,Census Tract 21.02,2041.000000,1204.000000,57.000000,19.000000,60.000000,2.0,696.000000,1378.000000,44135.0000,144600.00000,244.000000
4,01001020300,AL,Autauga County,Census Tract 203,3339.000000,2738.000000,498.000000,16.000000,27.000000,1.0,55.000000,2130.000000,46731.0000,79900.00000,478.000000


#### Join with 2017 Census data

Note: The [Census data dictionary](https://www.census.gov/data/developers/data-sets/acs-5year/data-notes.html) suggests that `-666666666` is a placeholder value for data that is not available:
```A "-" entry in the estimate column indicates that either no sample observations or too few sample observations were available to compute an estimate, or a ratio of medians cannot be calculated because one or both of the median estimates falls in the lowest interval or upper interval of an open-ended distribution.```

In [8]:
census_data_2017 = pd.read_csv(
    "../output/census_tracts.csv",
    dtype = { "geoid": str },
    na_values = [ -666666666 ]
).rename(
    columns = {"geoid": "GEOID" }
)

print(len(census_data_2017))
census_data_2017.head()

8281


,GEOID,name,total_population,total_population_25_over,median_income,median_home_value,educational_attainment,white_alone,black_alone,native_alone,asian_alone,native_hawaiian_pacific_islander,some_other_race_alone,two_or_more,hispanic_or_latino,city,metro_area
0,11001007503,"Census Tract 75.03, District of Columbia, Dist...",2454,1425.0,26250.0,345600.0,308.0,122.0,2278.0,0.0,0.0,0.0,0.0,17.0,37.0,Washington,Washington-Arlington-Alexandria
1,11001007601,"Census Tract 76.01, District of Columbia, Dist...",4855,3463.0,34840.0,255000.0,727.0,311.0,4292.0,0.0,0.0,13.0,0.0,41.0,198.0,Washington,Washington-Arlington-Alexandria
2,11001007709,"Census Tract 77.09, District of Columbia, Dist...",2524,1817.0,33750.0,250000.0,344.0,20.0,2280.0,0.0,0.0,0.0,0.0,130.0,94.0,Washington,Washington-Arlington-Alexandria
3,11001009508,"Census Tract 95.08, District of Columbia, Dist...",3691,2838.0,56404.0,356600.0,1008.0,211.0,2688.0,68.0,71.0,0.0,0.0,66.0,587.0,Washington,Washington-Arlington-Alexandria
4,11001009904,"Census Tract 99.04, District of Columbia, Dist...",2979,1526.0,30728.0,298600.0,252.0,52.0,2375.0,0.0,0.0,15.0,0.0,46.0,491.0,Washington,Washington-Arlington-Alexandria


In [9]:
census_merged = pd.merge(
    census_data_2017,
    census_data_2000, 
    on = "GEOID",
    how = "left",
    suffixes = [ "_17", "_00" ],
    validate = "1:1",
)

census_merged["median_home_value_00"] = census_merged["median_home_value_00"].replace(0, np.nan)

census_merged.head()

,GEOID,name,total_population_17,total_population_25_over_17,median_income_17,median_home_value_17,educational_attainment_17,white_alone_17,black_alone_17,native_alone_17,...,white_alone_00,black_alone_00,native_alone_00,asian_alone_00,native_hawaiian_pacific_islander_00,hispanic_or_latino_00,total_population_25_over_00,median_income_00,median_home_value_00,educational_attainment_00
0,11001007503,"Census Tract 75.03, District of Columbia, Dist...",2454,1425.0,26250.0,345600.0,308.0,122.0,2278.0,0.0,...,28.0,2447.0,3.0,7.0,1.0,26.0,1456.0,21402.0,99200.0,55.0
1,11001007601,"Census Tract 76.01, District of Columbia, Dist...",4855,3463.0,34840.0,255000.0,727.0,311.0,4292.0,0.0,...,108.0,4393.0,7.0,5.0,0.0,57.0,2998.0,32930.0,101300.0,249.0
2,11001007709,"Census Tract 77.09, District of Columbia, Dist...",2524,1817.0,33750.0,250000.0,344.0,20.0,2280.0,0.0,...,29.0,1899.0,7.0,8.0,0.0,35.0,1323.0,29332.0,111600.0,135.0
3,11001009508,"Census Tract 95.08, District of Columbia, Dist...",3691,2838.0,56404.0,356600.0,1008.0,211.0,2688.0,68.0,...,25.0,2762.0,5.0,17.0,0.0,41.0,2138.0,41972.0,136300.0,457.0
4,11001009904,"Census Tract 99.04, District of Columbia, Dist...",2979,1526.0,30728.0,298600.0,252.0,52.0,2375.0,0.0,...,12.0,1826.0,6.0,2.0,0.0,17.0,1111.0,26745.0,126300.0,43.0


In [10]:
census_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8281 entries, 0 to 8280
Data columns (total 31 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   GEOID                                8281 non-null   object 
 1   name                                 8281 non-null   object 
 2   total_population_17                  8281 non-null   int64  
 3   total_population_25_over_17          8281 non-null   float64
 4   median_income_17                     8160 non-null   float64
 5   median_home_value_17                 7936 non-null   float64
 6   educational_attainment_17            8281 non-null   float64
 7   white_alone_17                       8281 non-null   float64
 8   black_alone_17                       8281 non-null   float64
 9   native_alone_17                      8281 non-null   float64
 10  asian_alone_17                       8281 non-null   float64
 11  native_hawaiian_pacific_island

In [11]:
census_merged["city"].value_counts()

New York City    4700
Oakland           980
Washington        967
Atlanta           951
Baltimore         683
Name: city, dtype: int64

In [12]:
census_merged.to_csv(
    "../output/census_data_metro.csv",
    index = False
)

# The gentrification measure
It is based on two tests detailed [here](http://www.governing.com/gov-data/gentrification-report-methodology.html).

#### Test 1: does the tract qualify for gentrification?

* The tract had a population of at least 500 residents at the beginning and end of a decade and was located within a central city

* The tract’s median household income was in the bottom 40th percentile when compared to all tracts within its metro area at the beginning of the decade.

* The tract’s median home value was in the bottom 40th percentile when compared to all tracts within its metro area at the beginning of the decade.

#### Test 2: has it gentrified?

* An increase in a tract's educational attainment, as measured by the percentage of residents age 25 and over holding bachelor’s degrees, was in the top third of all tracts within a metro area.

* A tract’s median home value increased when adjusted for inflation.

* The percentage increase in a tract’s inflation-adjusted median home value was in the top third of all tracts within a metro area.

*Calculate the metro-level variables needed for these tests on a city-wide level:*

* 40th percentile household income threshold for metro level data 
* 40th percentile median home value for metro level
* bachelor's degree — top third for metro level
* inflation-adjusted median home value — top third for all metro level

### Educational attainment percentage change

In [13]:
census_merged["educational_attainment_pct_17"] = (
    100 * census_merged["educational_attainment_17"] /
    census_merged["total_population_25_over_17"]
)

census_merged["educational_attainment_pct_00"] = (
    100 * census_merged["educational_attainment_00"] / 
    census_merged["total_population_25_over_00"]
)

census_merged["educational_attainment_change"] = (
    census_merged["educational_attainment_pct_17"] - 
    census_merged["educational_attainment_pct_00"]
)

### Home value percentage change

Per [BLS CPI inflation rate](https://data.bls.gov/cgi-bin/cpicalc.pl?cost1=1.00&year1=200001&year2=201701), 1 dollar in January 2000 was worth $1.44 in January 2017.

In [14]:
inflation_rate  = 1.44

Now calculate the varios thresholds for our gentrification tests: 
* `40th percentile household income threshold for metro level data`
* `40th percentile median home value for metro level` 
* `inflation-adjusted median home value — top third percentile for all metro level`

In [15]:
census_merged["home_pct_change"] = (
    (
        census_merged["median_home_value_17"] - # current median home value
        (census_merged["median_home_value_00"] * inflation_rate) # previous median home value adjusted for inflation
    ) / 
    census_merged["median_home_value_17"] # current median home value 
)

### Calculate percentiles 

In [16]:
census_merged_grp = census_merged.groupby("metro_area")

metro_metrics = (
    pd.DataFrame({
        "metro_median_income_00_q40": census_merged_grp["median_income_00"].quantile(0.4),
        "metro_median_home_value_00_q40": census_merged_grp["median_home_value_00"].quantile(0.4),
        "educational_attainment_change_q66": census_merged_grp["educational_attainment_change"].quantile(2.0/3),
        "home_pct_change_q66": census_merged_grp["home_pct_change"].quantile(2.0/3)
    })
    .reset_index()
)

metro_metrics

,metro_area,metro_median_income_00_q40,metro_median_home_value_00_q40,educational_attainment_change_q66,home_pct_change_q66
0,Atlanta-Sandy Springs-Alpharetta,46630.915002,108139.99848,9.109797,0.049062
1,Baltimore-Columbia-Towson,43464.600000,106757.59722,10.744444,0.319395
2,New York-Newark-Jersey City,43583.600072,181600.00000,11.164452,0.375903
3,San Francisco-Oakland-Berkeley,56636.600000,290200.00416,12.191118,0.346429
4,Washington-Arlington-Alexandria,54471.999864,147300.17242,10.928943,0.390918


In [17]:
census_merged_with_metro = pd.merge(
    census_merged,
    metro_metrics,
    on = "metro_area",
    how = "left",
)

### Select only tracts in the cities of interest

In [18]:
CITY_CSVS = {
    "Atlanta": "../data/city_tracts/atlanta_census_reporter/acs2018_5yr_B03002_14000US13121008201.csv", 
    "Baltimore": "../data/city_tracts/baltimore_census_reporter/acs2018_5yr_B03002_14000US24510260700.csv",
    "New York City": "../data/city_tracts/new_york_census_reporter/acs2018_5yr_B03002_14000US36047990100.csv",
    "Oakland": "../data/city_tracts/oakland_census_reporter/acs2018_5yr_B03002_14000US06001403400.csv",
    "Washington":"../data/city_tracts/washington_census_reporter/acs2018_5yr_B03002_14000US11001007403.csv"
}

In [19]:
tracts_in_cities = (
    pd.concat([
        pd.read_csv(path, usecols = [ "geoid" ])
        for city, path in CITY_CSVS.items()
    ])
    .assign(
        GEOID = lambda df: df["geoid"].str.slice(7, None)
    )
    .drop(columns = [ "geoid" ])
)

print(len(tracts_in_cities))

tracts_in_cities.head()

2799


,GEOID
0,13089020100
1,13089020200
2,13089020300
3,13089020400
4,13089020500


In [20]:
cities_only = pd.merge(
    tracts_in_cities,
    census_merged_with_metro,
    on = "GEOID",
    how = "left",
    validate = "1:1",
)

print(len(cities_only))

cities_only.head()

2799


,GEOID,name,total_population_17,total_population_25_over_17,median_income_17,median_home_value_17,educational_attainment_17,white_alone_17,black_alone_17,native_alone_17,...,median_home_value_00,educational_attainment_00,educational_attainment_pct_17,educational_attainment_pct_00,educational_attainment_change,home_pct_change,metro_median_income_00_q40,metro_median_home_value_00_q40,educational_attainment_change_q66,home_pct_change_q66
0,13089020100,"Census Tract 201, DeKalb County, Georgia",2076,1455.0,174526.0,617300.0,1231.0,1885.0,36.0,0.0,...,378900.0000,966.000000,84.604811,71.344165,13.260646,0.116125,46630.915002,108139.99848,9.109797,0.049062
1,13089020200,"Census Tract 202, DeKalb County, Georgia",2045,1657.0,87889.0,548200.0,1271.0,1596.0,187.0,0.0,...,395014.9555,1362.201689,76.704888,78.568721,-1.863833,-0.037617,46630.915002,108139.99848,9.109797,0.049062
2,13089020300,"Census Tract 203, DeKalb County, Georgia",4151,2934.0,113088.0,470300.0,2344.0,3612.0,146.0,27.0,...,256174.8167,1710.151854,79.890934,68.615498,11.275435,0.215625,46630.915002,108139.99848,9.109797,0.049062
3,13089020400,"Census Tract 204, DeKalb County, Georgia",2574,1891.0,78681.0,433000.0,1584.0,2251.0,74.0,0.0,...,252024.7580,1181.242777,83.765204,64.731802,19.033402,0.161858,46630.915002,108139.99848,9.109797,0.049062
4,13089020500,"Census Tract 205, DeKalb County, Georgia",3305,2685.0,55208.0,239600.0,1220.0,1596.0,1286.0,0.0,...,87499.9969,206.659561,45.437616,9.695550,35.742066,0.474124,46630.915002,108139.99848,9.109797,0.049062


### Run gentrification tests described above

Create columns for every test that needs to be passed:

1. First whether they qualify
2. Whether they gentrified

In [21]:
# The tract had a population of at least 500 residents at the beginning and 
# end of a decade and was located within a central city
cities_only["low_population"] = (
    (cities_only["total_population_00"] < 500) | 
    (cities_only["total_population_17"] < 500)
)  

# The tract’s median household income was in the bottom 40th percentile when compared 
# to all tracts within its metro area at the beginning of the decade.
cities_only["median_home_value_00_qualifies"] = (
    cities_only["median_home_value_00"] < cities_only["metro_median_home_value_00_q40"]
)

# The tract’s median home value was in the bottom 40th percentile when compared 
# to all tracts within its metro area at the beginning of the decade.
cities_only["median_40th_income_00_qualifies"] = (
    cities_only["median_income_00"] < cities_only["metro_median_income_00_q40"]
)

In [22]:
# An increase in a tract"s educational attainment, as measured by the 
# percentage of residents age 25 and over who hold bachelor’s degrees, 
# was in the top third percentile of all tracts within a metro area.
cities_only["educational_increase_is_top_3rd"] = (
    cities_only["educational_attainment_change"] > cities_only["educational_attainment_change_q66"]
)

# The median home value increased, after accounting for inflation
cities_only["median_home_increased"] = (
    cities_only["median_home_value_17"] > (cities_only["median_home_value_00"] * inflation_rate)
)

# The percentage increase in a tract’s inflation-adjusted median home 
# value was in the top third percentile of all tracts within a metro area.
cities_only["median_home_increase_is_top_3rd"] = (
    cities_only["home_pct_change"] > cities_only["home_pct_change_q66"]
)

In [23]:
cities_only["eligible_for_gentrification"] = (
  (cities_only["low_population"] == False) &
  (cities_only["median_home_value_00_qualifies"] == True) &
  (cities_only["median_40th_income_00_qualifies"] == True)
)

cities_only["eligible_for_gentrification"].sum()

882

In [24]:
cities_only["gentrified"] = (
    (cities_only["eligible_for_gentrification"] == True) & 
    (cities_only["educational_increase_is_top_3rd"] == True) & 
    (cities_only["median_home_increased"] == True) & 
    (cities_only["median_home_increase_is_top_3rd"] == True)
)

cities_only["gentrified"].sum()

289

In [25]:
cities_only["eligible_not_gentrified_highpop"] = (
  (cities_only["eligible_for_gentrification"] == True) &
  (cities_only["gentrified"] == False)
)

cities_only["eligible_not_gentrified_highpop"].sum()

593

In [26]:
cities_only["not_eligible_highpop"] = (
  (cities_only["low_population"] == False) &
  (cities_only["eligible_for_gentrification"] == False) 
)

cities_only["not_eligible_highpop"].sum()

1831

In [27]:
(
    cities_only
    [[
        "city",
        "eligible_for_gentrification",
        "gentrified",
        "not_eligible_highpop",
        "low_population",
    ]]
    .groupby("city")
    .sum()
    .astype(int)
)

,eligible_for_gentrification,gentrified,not_eligible_highpop,low_population
city,,,,
Atlanta,72,23,64,3
Baltimore,169,40,30,1
New York City,455,138,1635,77
Oakland,83,29,27,4
Washington,103,59,75,1


## Calculate race/ethnicity percentage changes

In [28]:
RACE_ETHNICITY_GROUPS = [
    "white_alone",
    "black_alone",
    "asian_alone",
    "native_alone",
    "native_hawaiian_pacific_islander",
    "hispanic_or_latino",
]

for group in RACE_ETHNICITY_GROUPS:
    cities_only[f"pct_{group}_17"] = (100 * cities_only[f"{group}_17"] / cities_only["total_population_17"]).round(3)
    cities_only[f"pct_{group}_00"] = (100 * cities_only[f"{group}_00"] / cities_only["total_population_00"]).round(3)
    
    cities_only[f"pct_{group}_change"] = (cities_only[f"pct_{group}_17"] - cities_only[f"pct_{group}_00"]).round(3)

In [29]:
cities_only.to_csv(
    "../output/gentrification.csv",
    index = False
)

## Merge data with shapefiles

Shapefiles sourced here: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2019&layergroup=Census+Tracts

In [30]:
states = {
    "DC":"../data/censusTracts/states/tl_2019_11_DC_tract/tl_2019_11_tract.shp",
    "Georgia": "../data/censusTracts/states/tl_2019_13_georgia_tract/tl_2019_13_tract.shp", 
    "Maryland": "../data/censusTracts/states/tl_2019_24_maryland_tract/tl_2019_24_tract.shp",
    "California": "../data/censusTracts/states/tl_2019_06_california_tract/tl_2019_06_tract.shp",
    "New York": "../data/censusTracts/states/tl_2019_36_NY_tract/tl_2019_36_tract.shp",
}


shape_files = (
        pd.concat([
            gp.read_file(path)
            .drop(columns = [
                "STATEFP",
                "COUNTYFP",
                "TRACTCE",
                "NAMELSAD",
                "NAME",
                "FUNCSTAT",
                "ALAND",
                "AWATER",
                "MTFCC"

            ])
        for state, path in states.items()
    ])
)

shape_files.head()

,GEOID,INTPTLAT,INTPTLON,geometry
0,11001010900,+38.8132364,-077.0238475,"POLYGON ((-77.03919 38.80050, -77.03913 38.800..."
1,11001010400,+38.8512514,-077.0009048,"POLYGON ((-77.00919 38.83918, -77.00915 38.839..."
2,11001009903,+38.8883180,-076.9212121,"POLYGON ((-76.93005 38.88982, -76.92797 38.889..."
3,11001009904,+38.8851712,-076.9311353,"POLYGON ((-76.93577 38.88596, -76.93524 38.885..."
4,11001000902,+38.9285125,-077.1077517,"POLYGON ((-77.11975 38.93435, -77.11886 38.935..."


In [31]:
type(shape_files)

geopandas.geodataframe.GeoDataFrame

In [32]:
len(shape_files)

16529

In [33]:
merged_shape_files = gp.GeoDataFrame(
    pd.merge(
        cities_only,
        shape_files, 
        on = "GEOID",
        how = "left",
        validate = "1:1", 
    )
    .dropna(subset = [ "city" ])
    .replace([np.inf, -np.inf], np.nan)
)

print(len(cities_only), len(merged_shape_files))

2799 2799


In [34]:
type(merged_shape_files)

geopandas.geodataframe.GeoDataFrame

In [35]:
merged_shape_files.head()

,GEOID,name,total_population_17,total_population_25_over_17,median_income_17,median_home_value_17,educational_attainment_17,white_alone_17,black_alone_17,native_alone_17,...,pct_native_alone_change,pct_native_hawaiian_pacific_islander_17,pct_native_hawaiian_pacific_islander_00,pct_native_hawaiian_pacific_islander_change,pct_hispanic_or_latino_17,pct_hispanic_or_latino_00,pct_hispanic_or_latino_change,INTPTLAT,INTPTLON,geometry
0,13089020100,"Census Tract 201, DeKalb County, Georgia",2076,1455.0,174526.0,617300.0,1231.0,1885.0,36.0,0.0,...,-0.224,0.0,0.0,0.0,4.576,10.768,-6.192,+33.7998709,-084.3432667,"POLYGON ((-84.34830 33.79151, -84.34828 33.791..."
1,13089020200,"Census Tract 202, DeKalb County, Georgia",2045,1657.0,87889.0,548200.0,1271.0,1596.0,187.0,0.0,...,-0.546,0.0,0.0,0.0,5.281,3.092,2.189,+33.7756064,-084.3402667,"POLYGON ((-84.34898 33.77168, -84.34897 33.772..."
2,13089020300,"Census Tract 203, DeKalb County, Georgia",4151,2934.0,113088.0,470300.0,2344.0,3612.0,146.0,27.0,...,0.128,0.0,0.0,0.0,3.589,2.210,1.379,+33.7659856,-084.3295009,"POLYGON ((-84.33998 33.76486, -84.33942 33.764..."
3,13089020400,"Census Tract 204, DeKalb County, Georgia",2574,1891.0,78681.0,433000.0,1584.0,2251.0,74.0,0.0,...,-0.517,0.0,0.0,0.0,2.875,3.341,-0.466,+33.7660727,-084.3443392,"POLYGON ((-84.34922 33.76220, -84.34921 33.764..."
4,13089020500,"Census Tract 205, DeKalb County, Georgia",3305,2685.0,55208.0,239600.0,1220.0,1596.0,1286.0,0.0,...,-0.062,0.0,0.0,0.0,5.446,2.935,2.511,+33.7526586,-084.3416193,"POLYGON ((-84.34921 33.75115, -84.34919 33.752..."


### Prepare data for maps

- drop columns that don"t need to be visualized
- make a geojson file for each city


In [36]:
merged_shape_files.columns

Index(['GEOID', 'name', 'total_population_17', 'total_population_25_over_17',
       'median_income_17', 'median_home_value_17', 'educational_attainment_17',
       'white_alone_17', 'black_alone_17', 'native_alone_17', 'asian_alone_17',
       'native_hawaiian_pacific_islander_17', 'some_other_race_alone',
       'two_or_more', 'hispanic_or_latino_17', 'city', 'metro_area', 'state',
       'county', 'tract', 'total_population_00', 'white_alone_00',
       'black_alone_00', 'native_alone_00', 'asian_alone_00',
       'native_hawaiian_pacific_islander_00', 'hispanic_or_latino_00',
       'total_population_25_over_00', 'median_income_00',
       'median_home_value_00', 'educational_attainment_00',
       'educational_attainment_pct_17', 'educational_attainment_pct_00',
       'educational_attainment_change', 'home_pct_change',
       'metro_median_income_00_q40', 'metro_median_home_value_00_q40',
       'educational_attainment_change_q66', 'home_pct_change_q66',
       'low_population', 

In [37]:
columns_for_viz = [
    "GEOID",
    "name",
    "total_population_17",
    "white_alone_17",
    "black_alone_17",
    "native_alone_17",
    "asian_alone_17",
    "native_hawaiian_pacific_islander_17",
    "hispanic_or_latino_17",
    "city",
    "gentrified",
    "pct_white_alone_change",
    "pct_black_alone_change",
    "pct_native_alone_change",
    "pct_asian_alone_change",
    "pct_hispanic_or_latino_change",
    "pct_native_hawaiian_pacific_islander_change",
    "INTPTLAT", "INTPTLON", "geometry"
]

merged_shape_files_for_viz = merged_shape_files[columns_for_viz]

In [38]:
city_list = merged_shape_files_for_viz["city"].unique().tolist()

for city in city_list:
    print(city)
    output_path = "../output/for_maps/" + str(city) + "_data.csv"
    output_path_geojson = "../output/for_maps/" + str(city) + "_data.geojson"
    
    df = merged_shape_files_for_viz[merged_shape_files_for_viz["city"] == city]
    df.drop(columns = [ "geometry" ]).to_csv(output_path, index = False)
    df.to_file(
        output_path_geojson,
        driver = "GeoJSON"
    )

Atlanta
Baltimore
New York City
Oakland
Washington


---

---

---